In [95]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier

In [96]:
years = [x for x in range(2000, 2025, 1)] # For loading data with ease

In [97]:
max(years) # Checking if max year is correct

2024

In [98]:
X_train = pd.DataFrame() # Created an empty dataframe to begin

In [99]:
# Load data with a for loop to avoid redundancy
for year in years:
    new = pd.read_csv(f"atp_matches_{year}.csv")
    X_train = pd.concat([X_train, new], ignore_index=True) # Concatenate to main dataset

In [100]:
X_train

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2000-301,Auckland,Hard,32,A,20000110,1,103163,1.0,NaN,...,55.0,39.0,29.0,17.0,4.0,7.0,11.0,1612.0,63.0,595.0
1,2000-301,Auckland,Hard,32,A,20000110,2,102607,NaN,Q,...,32.0,25.0,18.0,12.0,3.0,6.0,211.0,157.0,49.0,723.0
2,2000-301,Auckland,Hard,32,A,20000110,3,103252,NaN,NaN,...,33.0,20.0,7.0,8.0,7.0,11.0,48.0,726.0,59.0,649.0
3,2000-301,Auckland,Hard,32,A,20000110,4,103507,7.0,NaN,...,43.0,29.0,14.0,10.0,6.0,8.0,45.0,768.0,61.0,616.0
4,2000-301,Auckland,Hard,32,A,20000110,5,102103,NaN,Q,...,46.0,34.0,18.0,12.0,5.0,9.0,167.0,219.0,34.0,873.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74901,2024-M-DC-2024-WG2-PO-URU-MDA-01,Davis Cup WG2 PO: URU vs MDA,Clay,4,D,20240203,5,212051,NaN,NaN,...,30.0,17.0,7.0,6.0,8.0,14.0,1109.0,8.0,740.0,34.0
74902,2024-M-DC-2024-WG2-PO-VIE-RSA-01,Davis Cup WG2 PO: VIE vs RSA,Hard,4,D,20240202,1,122533,NaN,NaN,...,41.0,25.0,6.0,9.0,1.0,4.0,554.0,67.0,748.0,32.0
74903,2024-M-DC-2024-WG2-PO-VIE-RSA-01,Davis Cup WG2 PO: VIE vs RSA,Hard,4,D,20240202,2,144748,NaN,NaN,...,51.0,25.0,7.0,11.0,5.0,12.0,416.0,109.0,NaN,NaN
74904,2024-M-DC-2024-WG2-PO-VIE-RSA-01,Davis Cup WG2 PO: VIE vs RSA,Hard,4,D,20240202,4,122533,NaN,NaN,...,51.0,32.0,17.0,14.0,5.0,9.0,554.0,67.0,416.0,109.0


In [101]:
X_train.shape

(74906, 49)

In [102]:
X_train.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points'],
      dtype='object')

In [103]:
X_train.describe()

,draw_size,tourney_date,match_num,winner_id,winner_seed,winner_ht,winner_age,loser_id,loser_seed,loser_ht,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
count,74906.000000,7.490600e+04,74906.000000,74906.000000,31120.000000,73481.000000,74901.000000,74906.000000,17238.000000,71997.000000,...,68386.000000,68386.000000,68386.000000,68386.000000,68386.000000,68386.000000,74333.000000,74333.000000,73438.000000,73438.000000
mean,55.345126,2.011551e+07,102.269164,111056.333818,7.402635,186.273363,26.289788,111055.887299,8.920931,185.671570,...,48.635481,32.474790,14.929576,12.308104,4.771898,8.601468,79.644250,1608.435755,117.699965,977.409039
std,40.315190,7.240072e+04,133.184752,22766.835939,6.815177,7.028887,3.980862,22671.363557,7.366649,6.979333,...,19.198909,14.358641,7.185611,4.220819,3.271866,4.145550,138.795266,2006.339266,185.938285,1125.591169
min,2.000000,2.000010e+07,1.000000,100644.000000,1.000000,3.000000,14.900000,100644.000000,1.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,32.000000,2.005080e+07,12.000000,103565.000000,3.000000,183.000000,23.400000,103490.000000,4.000000,181.000000,...,35.000000,22.000000,10.000000,9.000000,2.000000,6.000000,18.000000,577.000000,36.000000,430.000000
50%,32.000000,2.011072e+07,30.000000,104494.000000,5.000000,185.000000,26.100000,104471.000000,7.000000,185.000000,...,45.000000,30.000000,14.000000,11.000000,4.000000,8.000000,45.000000,940.000000,68.000000,710.000000
75%,64.000000,2.018012e+07,220.000000,105493.000000,9.000000,190.000000,28.900000,105613.000000,12.000000,190.000000,...,59.000000,40.000000,19.000000,15.000000,7.000000,11.000000,85.000000,1732.000000,114.000000,1105.000000
max,128.000000,2.024122e+07,1701.000000,212721.000000,35.000000,211.000000,44.600000,212970.000000,35.000000,211.000000,...,328.000000,284.000000,101.000000,91.000000,27.000000,38.000000,2101.000000,16950.000000,2159.000000,16950.000000


In [104]:
X_train.isna().sum()

tourney_id                0
tourney_name              0
surface                  53
draw_size                 0
tourney_level             0
tourney_date              0
match_num                 0
winner_id                 0
winner_seed           43786
winner_entry          65400
winner_name               0
winner_hand               0
winner_ht              1425
winner_ioc                0
winner_age                5
loser_id                  0
loser_seed            57668
loser_entry           59484
loser_name                0
loser_hand                4
loser_ht               2909
loser_ioc                 0
loser_age                 3
score                     0
best_of                   0
round                     0
minutes                8174
w_ace                  6520
w_df                   6520
w_svpt                 6520
w_1stIn                6520
w_1stWon               6520
w_2ndWon               6520
w_SvGms                6520
w_bpSaved              6520
w_bpFaced           

In [105]:
X_train.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points'],
      dtype='object')

In [106]:
# Defining what columns to drop
columns_to_drop = [
    'tourney_id',
    'match_num',
    'winner_id',
    'winner_entry',
    'loser_id',
    'loser_entry',
]

In [107]:
X_train.drop(columns=columns_to_drop, inplace=True, axis=1) # Drop unnecessary columns

In [108]:
X_train

,tourney_name,surface,draw_size,tourney_level,tourney_date,winner_seed,winner_name,winner_hand,winner_ht,winner_ioc,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,Auckland,Hard,32,A,20000110,1.0,Tommy Haas,R,188.0,GER,...,55.0,39.0,29.0,17.0,4.0,7.0,11.0,1612.0,63.0,595.0
1,Auckland,Hard,32,A,20000110,NaN,Juan Balcells,R,190.0,ESP,...,32.0,25.0,18.0,12.0,3.0,6.0,211.0,157.0,49.0,723.0
2,Auckland,Hard,32,A,20000110,NaN,Alberto Martin,R,175.0,ESP,...,33.0,20.0,7.0,8.0,7.0,11.0,48.0,726.0,59.0,649.0
3,Auckland,Hard,32,A,20000110,7.0,Juan Carlos Ferrero,R,183.0,ESP,...,43.0,29.0,14.0,10.0,6.0,8.0,45.0,768.0,61.0,616.0
4,Auckland,Hard,32,A,20000110,NaN,Michael Sell,R,180.0,USA,...,46.0,34.0,18.0,12.0,5.0,9.0,167.0,219.0,34.0,873.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74901,Davis Cup WG2 PO: URU vs MDA,Clay,4,D,20240203,NaN,Joaquin Aguilar Cardozo,U,NaN,URU,...,30.0,17.0,7.0,6.0,8.0,14.0,1109.0,8.0,740.0,34.0
74902,Davis Cup WG2 PO: VIE vs RSA,Hard,4,D,20240202,NaN,Nam Hoang Ly,R,175.0,VIE,...,41.0,25.0,6.0,9.0,1.0,4.0,554.0,67.0,748.0,32.0
74903,Davis Cup WG2 PO: VIE vs RSA,Hard,4,D,20240202,NaN,Kris Van Wyk,R,185.0,RSA,...,51.0,25.0,7.0,11.0,5.0,12.0,416.0,109.0,NaN,NaN
74904,Davis Cup WG2 PO: VIE vs RSA,Hard,4,D,20240202,NaN,Nam Hoang Ly,R,175.0,VIE,...,51.0,32.0,17.0,14.0,5.0,9.0,554.0,67.0,416.0,109.0


In [109]:
from sklearn.model_selection import train_test_split

In [110]:
# Split data
X_train, X_test = train_test_split(X_train, test_size=0.2, random_state=42)

In [111]:
X_train

,tourney_name,surface,draw_size,tourney_level,tourney_date,winner_seed,winner_name,winner_hand,winner_ht,winner_ioc,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
28872,Beijing Olympics,Hard,64,A,20080811,NaN,Agustin Calleri,R,183.0,ARG,...,38.0,18.0,5.0,7.0,7.0,12.0,52.0,765.0,1017.0,8.0
64262,Doha,Hard,32,A,20210308,5.0,Roberto Bautista Agut,R,183.0,ESP,...,53.0,42.0,19.0,14.0,9.0,11.0,13.0,2770.0,39.0,1457.0
65928,Atp Cup,Hard,16,A,20220103,NaN,Denis Shapovalov,L,185.0,CAN,...,33.0,21.0,8.0,9.0,3.0,6.0,14.0,2475.0,20.0,2230.0
24753,Sopot,Clay,32,A,20070730,2.0,Tommy Robredo,R,180.0,ESP,...,45.0,27.0,11.0,14.0,2.0,9.0,7.0,2085.0,134.0,320.0
33441,Barcelona,Clay,56,A,20100419,NaN,Ernests Gulbis,R,190.0,LAT,...,30.0,20.0,9.0,8.0,4.0,8.0,44.0,939.0,46.0,920.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37194,Davis Cup G1 R2: SUI vs POR,Hard,4,D,20110708,NaN,Stan Wawrinka,R,183.0,SUI,...,NaN,NaN,NaN,NaN,NaN,NaN,15.0,1935.0,311.0,141.0
6265,Bucharest,Clay,32,A,20010910,6.0,Jerome Golmard,L,188.0,FRA,...,37.0,26.0,17.0,12.0,2.0,6.0,48.0,720.0,76.0,542.0
54886,Wimbledon,Grass,128,G,20170703,NaN,Dusan Lajovic,R,183.0,SRB,...,49.0,39.0,18.0,15.0,5.0,8.0,79.0,661.0,192.0,286.0
860,Miami Masters,Hard,96,M,20000320,NaN,Agustin Calleri,R,183.0,ARG,...,50.0,36.0,34.0,16.0,5.0,8.0,124.0,327.0,46.0,796.0


In [112]:
X_test

,tourney_name,surface,draw_size,tourney_level,tourney_date,winner_seed,winner_name,winner_hand,winner_ht,winner_ioc,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
48014,Indian Wells Masters,Hard,96,M,20150312,8.0,David Ferrer,R,175.0,ESP,...,52.0,32.0,28.0,14.0,5.0,9.0,8.0,4535.0,96.0,560.0
46119,Wimbledon,Grass,128,G,20140623,NaN,Lukasz Kubot,R,191.0,POL,...,80.0,61.0,32.0,22.0,1.0,3.0,72.0,713.0,84.0,650.0
48076,Miami Masters,Hard,96,M,20150325,NaN,Alexander Zverev,R,198.0,GER,...,59.0,53.0,16.0,17.0,1.0,3.0,129.0,432.0,69.0,701.0
74071,Us Open,Hard,128,G,20240826,NaN,Rinky Hijikata,R,178.0,AUS,...,92.0,57.0,23.0,20.0,8.0,16.0,62.0,848.0,59.0,900.0
13446,Amersfoort,Clay,32,A,20040712,NaN,Alejandro Falla,L,185.0,COL,...,30.0,16.0,9.0,8.0,2.0,7.0,118.0,352.0,39.0,831.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40585,Washington,Hard,32,A,20120730,NaN,Florent Serra,R,180.0,FRA,...,48.0,31.0,17.0,14.0,4.0,8.0,132.0,424.0,78.0,625.0
22186,US Open,Hard,128,G,20060828,1.0,Roger Federer,R,185.0,SUI,...,80.0,53.0,16.0,18.0,10.0,16.0,1.0,7295.0,10.0,1890.0
54172,Monte Carlo Masters,Clay,64,M,20170417,10.0,David Goffin,R,180.0,BEL,...,29.0,16.0,10.0,8.0,8.0,13.0,13.0,2705.0,51.0,899.0
71613,Davis Cup Finals RR: CAN vs ITA,Hard,2,D,20230913,NaN,Gabriel Diallo,R,203.0,CAN,...,39.0,29.0,10.0,11.0,1.0,3.0,158.0,394.0,18.0,1925.0


In [113]:
X_train.columns

Index(['tourney_name', 'surface', 'draw_size', 'tourney_level', 'tourney_date',
       'winner_seed', 'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc',
       'winner_age', 'loser_seed', 'loser_name', 'loser_hand', 'loser_ht',
       'loser_ioc', 'loser_age', 'score', 'best_of', 'round', 'minutes',
       'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms',
       'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt', 'l_1stIn',
       'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points'],
      dtype='object')

In [114]:
np.random.seed(42)
shuffle_mask = np.random.rand(len(X_train)) < 0.5

X_train['player_1'] = np.where(shuffle_mask, X_train['winner_name'], X_train['loser_name'])
X_train['player_2'] = np.where(shuffle_mask, X_train['loser_name'], X_train['winner_name'])

# Step 2: Binary target
X_train['player_1_won'] = (X_train['player_1'] == X_train['winner_name']).astype(int)

# Step 3: Stats features
stats_cols = ['ace', 'df', 'svpt', '1stIn', '1stWon', '2ndWon', 'SvGms', 'bpSaved', 'bpFaced']
for col in stats_cols:
    X_train[f'player_1_{col}'] = np.where(shuffle_mask, X_train[f'w_{col}'], X_train[f'l_{col}'])
    X_train[f'player_2_{col}'] = np.where(shuffle_mask, X_train[f'l_{col}'], X_train[f'w_{col}'])

# Step 4: Keep only features we want
feature_cols = ['tourney_name', 'surface', 'draw_size', 'tourney_level', 'tourney_date', 'best_of', 'round', 'minutes'] + \
               [f'player_1_{c}' for c in stats_cols] + [f'player_2_{c}' for c in stats_cols]

X = X_train[feature_cols]
y = X_train['player_1_won']

# Step 5: Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train distribution:\n", y_train.value_counts())

X_train shape: (47939, 26)
X_test shape: (11985, 26)
y_train distribution:
 player_1_won
0    23981
1    23958
Name: count, dtype: int64


In [115]:
from sklearn.impute import SimpleImputer # For filling nan

In [116]:
imputer = SimpleImputer().fit(X_train.select_dtypes(include=np.number)) # Fit on training data to avoid data leakage

In [117]:
numeric_cols = X_train.select_dtypes(include=np.number).columns # For easier use later on

In [118]:
numeric_cols

Index(['draw_size', 'tourney_date', 'best_of', 'minutes', 'player_1_ace',
       'player_1_df', 'player_1_svpt', 'player_1_1stIn', 'player_1_1stWon',
       'player_1_2ndWon', 'player_1_SvGms', 'player_1_bpSaved',
       'player_1_bpFaced', 'player_2_ace', 'player_2_df', 'player_2_svpt',
       'player_2_1stIn', 'player_2_1stWon', 'player_2_2ndWon',
       'player_2_SvGms', 'player_2_bpSaved', 'player_2_bpFaced'],
      dtype='object')

In [119]:
# Transform numeric columns and remove nans
X_train[numeric_cols] = imputer.transform(X_train[numeric_cols])
X_test[numeric_cols] = imputer.transform(X_test[numeric_cols])

In [120]:
X_train[numeric_cols].isna().sum()

draw_size           0
tourney_date        0
best_of             0
minutes             0
player_1_ace        0
player_1_df         0
player_1_svpt       0
player_1_1stIn      0
player_1_1stWon     0
player_1_2ndWon     0
player_1_SvGms      0
player_1_bpSaved    0
player_1_bpFaced    0
player_2_ace        0
player_2_df         0
player_2_svpt       0
player_2_1stIn      0
player_2_1stWon     0
player_2_2ndWon     0
player_2_SvGms      0
player_2_bpSaved    0
player_2_bpFaced    0
dtype: int64

In [121]:
X_test[numeric_cols].isna().sum()

draw_size           0
tourney_date        0
best_of             0
minutes             0
player_1_ace        0
player_1_df         0
player_1_svpt       0
player_1_1stIn      0
player_1_1stWon     0
player_1_2ndWon     0
player_1_SvGms      0
player_1_bpSaved    0
player_1_bpFaced    0
player_2_ace        0
player_2_df         0
player_2_svpt       0
player_2_1stIn      0
player_2_1stWon     0
player_2_2ndWon     0
player_2_SvGms      0
player_2_bpSaved    0
player_2_bpFaced    0
dtype: int64

In [122]:
from sklearn.preprocessing import OneHotEncoder # To encode categorical columns

In [123]:
# Get the categorical columns for encoding
categorical_cols = X_train.select_dtypes(include='object').columns

In [124]:
categorical_cols

Index(['tourney_name', 'surface', 'tourney_level', 'round'], dtype='object')

In [125]:
enc = OneHotEncoder(handle_unknown="ignore").fit(X_train[categorical_cols]) # Fit encoder on training dataset to avoid data leakage

In [126]:
X_train.columns

Index(['tourney_name', 'surface', 'draw_size', 'tourney_level', 'tourney_date',
       'best_of', 'round', 'minutes', 'player_1_ace', 'player_1_df',
       'player_1_svpt', 'player_1_1stIn', 'player_1_1stWon', 'player_1_2ndWon',
       'player_1_SvGms', 'player_1_bpSaved', 'player_1_bpFaced',
       'player_2_ace', 'player_2_df', 'player_2_svpt', 'player_2_1stIn',
       'player_2_1stWon', 'player_2_2ndWon', 'player_2_SvGms',
       'player_2_bpSaved', 'player_2_bpFaced'],
      dtype='object')

In [127]:
X_test.columns

Index(['tourney_name', 'surface', 'draw_size', 'tourney_level', 'tourney_date',
       'best_of', 'round', 'minutes', 'player_1_ace', 'player_1_df',
       'player_1_svpt', 'player_1_1stIn', 'player_1_1stWon', 'player_1_2ndWon',
       'player_1_SvGms', 'player_1_bpSaved', 'player_1_bpFaced',
       'player_2_ace', 'player_2_df', 'player_2_svpt', 'player_2_1stIn',
       'player_2_1stWon', 'player_2_2ndWon', 'player_2_SvGms',
       'player_2_bpSaved', 'player_2_bpFaced'],
      dtype='object')

In [128]:
# Transform categories
X_train_encoded = pd.DataFrame(
    enc.transform(X_train[categorical_cols]).toarray(), 
    columns=enc.get_feature_names_out(categorical_cols),
    index=X_train.index
)

# Transform categories
X_test_encoded = pd.DataFrame(
    enc.transform(X_test[categorical_cols]).toarray(), 
    columns=enc.get_feature_names_out(categorical_cols),
    index=X_test.index
)

# Drop old categorical columns and concat encoded ones
X_train = X_train.drop(columns=categorical_cols).join(X_train_encoded)
X_test = X_test.drop(columns=categorical_cols).join(X_test_encoded)

In [129]:
X_train

,draw_size,tourney_date,best_of,minutes,player_1_ace,player_1_df,player_1_svpt,player_1_1stIn,player_1_1stWon,player_1_2ndWon,...,round_BR,round_ER,round_F,round_QF,round_R128,round_R16,round_R32,round_R64,round_RR,round_SF
62746,128.0,20200928.0,5.0,170.000000,10.000000,2.000000,134.000000,87.000000,57.000000,20.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
61533,8.0,20191111.0,3.0,100.000000,5.000000,0.000000,68.000000,47.000000,33.000000,10.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
58682,64.0,20181008.0,3.0,90.000000,7.000000,2.000000,63.000000,41.000000,32.000000,13.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
31693,128.0,20090119.0,5.0,137.000000,9.000000,5.000000,118.000000,63.000000,41.000000,29.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
68901,32.0,20230102.0,3.0,67.000000,0.000000,0.000000,42.000000,31.000000,13.000000,6.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33272,96.0,20100324.0,3.0,92.000000,6.000000,2.000000,79.000000,46.000000,36.000000,17.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6459,4.0,20010209.0,5.0,107.269656,6.025754,2.994493,79.544676,48.397738,34.458684,15.742847,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
60203,128.0,20190527.0,5.0,150.000000,16.000000,7.000000,129.000000,76.000000,60.000000,20.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
39911,128.0,20120527.0,5.0,229.000000,2.000000,5.000000,147.000000,92.000000,55.000000,27.000000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [130]:
X_train.drop('tourney_date', inplace=True, axis=1)
X_test.drop('tourney_date', inplace=True, axis=1)

In [131]:
X_train.columns

Index(['draw_size', 'best_of', 'minutes', 'player_1_ace', 'player_1_df',
       'player_1_svpt', 'player_1_1stIn', 'player_1_1stWon', 'player_1_2ndWon',
       'player_1_SvGms',
       ...
       'round_BR', 'round_ER', 'round_F', 'round_QF', 'round_R128',
       'round_R16', 'round_R32', 'round_R64', 'round_RR', 'round_SF'],
      dtype='object', length=1946)

In [132]:
X_test.columns

Index(['draw_size', 'best_of', 'minutes', 'player_1_ace', 'player_1_df',
       'player_1_svpt', 'player_1_1stIn', 'player_1_1stWon', 'player_1_2ndWon',
       'player_1_SvGms',
       ...
       'round_BR', 'round_ER', 'round_F', 'round_QF', 'round_R128',
       'round_R16', 'round_R32', 'round_R64', 'round_RR', 'round_SF'],
      dtype='object', length=1946)

In [133]:
X_test

,draw_size,best_of,minutes,player_1_ace,player_1_df,player_1_svpt,player_1_1stIn,player_1_1stWon,player_1_2ndWon,player_1_SvGms,...,round_BR,round_ER,round_F,round_QF,round_R128,round_R16,round_R32,round_R64,round_RR,round_SF
7062,32.0,3.0,97.000000,0.0,0.0,68.0,47.0,33.0,11.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
47006,56.0,3.0,150.000000,12.0,3.0,95.0,58.0,46.0,16.0,17.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7996,64.0,3.0,108.000000,8.0,8.0,98.0,61.0,44.0,18.0,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
65216,32.0,3.0,143.000000,4.0,0.0,101.0,58.0,44.0,26.0,16.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
39257,96.0,3.0,158.000000,2.0,2.0,97.0,68.0,41.0,12.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14803,32.0,3.0,79.000000,1.0,4.0,68.0,43.0,24.0,12.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
30180,96.0,3.0,108.000000,3.0,0.0,74.0,42.0,27.0,15.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
49338,28.0,3.0,107.269656,5.0,3.0,51.0,27.0,19.0,10.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
34887,28.0,3.0,165.000000,7.0,1.0,115.0,92.0,69.0,10.0,16.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [149]:
# Fit XGBoost
model = XGBClassifier(
    n_jobs=-1,
    max_depth=2,
    random_state=42,
    n_estimators=1000
).fit(
    X_train, 
    y_train, 
    eval_set=[(X_test, y_test)],
    verbose=1
)

[0]	validation_0-logloss:0.61630
[1]	validation_0-logloss:0.56849
[2]	validation_0-logloss:0.53828
[3]	validation_0-logloss:0.51783
[4]	validation_0-logloss:0.50192
[5]	validation_0-logloss:0.48773
[6]	validation_0-logloss:0.47258
[7]	validation_0-logloss:0.45754
[8]	validation_0-logloss:0.44911
[9]	validation_0-logloss:0.44003
[10]	validation_0-logloss:0.42987
[11]	validation_0-logloss:0.42384
[12]	validation_0-logloss:0.41870
[13]	validation_0-logloss:0.41213
[14]	validation_0-logloss:0.40490
[15]	validation_0-logloss:0.39800
[16]	validation_0-logloss:0.39272
[17]	validation_0-logloss:0.38885
[18]	validation_0-logloss:0.38483
[19]	validation_0-logloss:0.37974
[20]	validation_0-logloss:0.37428
[21]	validation_0-logloss:0.37037
[22]	validation_0-logloss:0.36471
[23]	validation_0-logloss:0.36004
[24]	validation_0-logloss:0.35562
[25]	validation_0-logloss:0.34948
[26]	validation_0-logloss:0.34634
[27]	validation_0-logloss:0.34296
[28]	validation_0-logloss:0.33918
[29]	validation_0-loglos

In [150]:
from sklearn.metrics import accuracy_score

In [153]:
# Predict classes
y_pred = model.predict(X_train)

# Calculate accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_train, y_pred) * 100
print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 92.47%


In [152]:
# Predict classes
y_pred = model.predict(X_test)

# Calculate accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred) * 100
print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 91.15%
